In [1]:
import numpy as np
import pandas as pd
from data_utils import process_data, get_device
import torch

In [2]:
df = pd.read_csv("dataset/BA_weather_6-23_jan.csv", parse_dates=["Date"])

In [3]:
weather_data = df.iloc[::-1]
weather_data.reset_index(inplace=True)


In [4]:
weather_data

,index,Date,T,P0,U,Ff
0,861,2023-06-01 00:00:00,9,753.9,71,4
1,860,2023-06-01 00:30:00,9,753.9,71,4
2,859,2023-06-01 01:00:00,9,753.9,71,4
3,858,2023-06-01 01:30:00,8,754.7,76,4
4,857,2023-06-01 02:00:00,8,754.7,76,3
...,...,...,...,...,...,...
857,4,2023-01-23 21:30:00,2,759.6,87,3
858,3,2023-01-23 22:00:00,1,759.6,100,3
859,2,2023-01-23 22:30:00,1,760.4,93,3
860,1,2023-01-23 23:00:00,2,760.4,87,2


In [5]:
X, y = process_data(weather_data)

/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["day_of_year"] = weather_data["Date"].dt.dayofyear
/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["time"] = pd.to_datetime(
/Users/anmacko/Documents/FMFI/ML/WeatherProphet/data_utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [6]:
mean, std = np.load("dataset/stat.npy")

In [7]:
X = (X - mean) / std

In [8]:
X

,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,P0,T,U,Ff
0,0.724488,-1.241554,-0.000427,1.415490,0.544631,-0.296610,-0.033131,0.110761
2,0.724488,-1.241554,0.365557,1.367297,0.544631,-0.296610,-0.033131,0.110761
4,0.724488,-1.241554,0.706600,1.226002,0.683916,-0.409101,0.243570,-0.334331
6,0.724488,-1.241554,0.999460,1.001235,0.683916,-0.296610,-0.033131,-0.334331
8,0.724488,-1.241554,1.224180,0.708312,0.683916,-0.409101,0.243570,-0.334331
...,...,...,...,...,...,...,...,...
852,0.560688,1.282359,-1.366298,0.367197,1.554444,-0.971558,0.852314,-0.779423
854,0.560688,1.282359,-1.225034,0.708312,1.554444,-0.971558,0.852314,-0.779423
856,0.560688,1.282359,-1.000314,1.001235,1.537033,-1.196540,1.184355,-0.334331
858,0.560688,1.282359,-0.707454,1.226002,1.537033,-1.196540,1.571737,-0.334331


In [9]:
y

0      NaN
2      9.0
4      9.0
6      8.0
8      9.0
      ... 
852    3.0
854    3.0
856    3.0
858    1.0
860    1.0
Name: next_T, Length: 431, dtype: float64

In [10]:
X, y = X.to_numpy()[1:], y.to_numpy()[1:]

In [11]:
x1 = X[:6]
y1 = y[6]


In [12]:
x1.shape

(6, 8)

In [23]:
x1 = np.reshape(x1, (1,6,8))

In [15]:
y1

8.0

In [16]:
device = get_device()


In [17]:
wp = torch.load('WP100-wtf_multistep3.pt')

In [18]:
wp.to(device)

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [19]:
wp.eval()

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [24]:
X_features = torch.from_numpy(x1).float().to(device)

In [25]:
X_features.shape

torch.Size([1, 6, 8])

In [26]:
y_hat = wp(X_features)

In [27]:
y_hat

/opt/anaconda3/envs/masters/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646991213/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[[7.8834]]], device='mps:0', grad_fn=<LinearBackward0>)